In [1]:
import time

st = time.time()

In [2]:
import io
import pandas as pd
import tensorflow as tf
from PIL import Image as PIL_Image
from img2table.document import Image
from pdf2image import convert_from_path
import functions as fns  # For concise function calls

In [3]:
# Dictionary for storing marks of each papers
my_dict = {'1a': [], '1b': [], '1c': [], '2a': [], '2b': [], '2c': [], '3a': [], '3b': [], '3c': [], '4a': [], '4b': [], '4c': [], '5a': [], '5b': [], '5c': [], '6a': [], '6b': [], '6c': [], '7a': [], '7b': [], '7c': [], '8a': [], '8b': [], '8c': [], '9a': [], '9b': [], '9c': [], '10a': [], '10b': [], '10c': [], '11a': [], '11b': [], '11c': [], '12a': [], '12b': [], '12c': []}


In [4]:
model = tf.keras.models.load_model("D:/AJAYMON/AJAY/Programming/S6_Mini_Project/Codes/Neural Network/MP_Latest_Model.h5")

images = convert_from_path("D:/AJAYMON/AJAY/Programming/S6_Mini_Project/Codes/Image 2 Table/test_set.pdf")

In [5]:
for i in range(len(images)):
    try:
        img = images[i]
        _, height = img.size
        cropped_img = img.crop((150, height / 2 + 50, 1600, height - 300)) # left, top, right, bottom
        # cropped_img.save(f"D:/AJAYMON/AJAY/Programming/Auto_Excel_Mark_Entry/Codes/image to table/temp/ima{i}.jpg") # To save cropped image
        
        dpi=(200,200)
        img.info["dpi"]=dpi
        img_bytes=io.BytesIO()
        cropped_img.save(img_bytes, format='JPEG')
        img_bytes.seek(0)

        doc = Image(img_bytes)
        extracted_tables = doc.extract_tables(implicit_rows=False, min_confidence=50)
        
        orddict = extracted_tables[0].content
        
        img=PIL_Image.open(img_bytes)
        
        if len(orddict.keys()) == 5 and sum(len(value) for value in orddict.values()) == 65:# if table recognition is correct 
            # print(f"At if {i}")                                                           # then, the no of rows & columns
            paper_df = fns.cell_extraction_classification_df_return(orddict, img, model)    # will be 5 & 65 respectively
            marks_for_main_dict = fns.dataframe_postprocessing(paper_df)
        
            for key, value in zip(my_dict.keys(), marks_for_main_dict):
                my_dict[key].append(value) # Adding values to dictionary
        
        else: # if table recognition is incorrect
            # print(f"At else {i}")
            for key in my_dict.keys():
                my_dict[key].append(0) # Adding values to dictionary
    except:
        # print(f"At else {i}") # if table recognition is incorrect
        for key in my_dict.keys():
            my_dict[key].append(0) # Adding values to dictionary            

1/1 [==============================] - 0s 24ms/step


In [6]:
dict_df = pd.DataFrame(my_dict) 

columns_to_drop = [col for col in dict_df.columns if dict_df[col].nunique() == 1 and dict_df[col].unique()[0] == 0] # removing columns with 0 from start to end
dict_df.drop(columns=columns_to_drop, inplace=True) # dropping the 0s columns (modifys the base table with 'inplace=True')

dict_df['Sum'] = dict_df.sum(axis=1)

dict_df.insert(0, 'Name', '') # Add empty colum, Name to left of df
dict_df.insert(0, 'Roll No', '')

dict_df.to_csv("D:/AJAYMON/AJAY/Programming/S6_Mini_Project/Codes/Image 2 Table/test_set.csv", index=False)

In [7]:
print(f"Total time: {round(((time.time() - st) / 60), 3)} minutes")

Total time: 0.281 minutes
